In [3]:
from selenium import webdriver
import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.remote.webelement headless WebElement
from sys import platform

import os
import chromedriver_autoinstaller
import pandas as pd
import json
import csv
from datetime import datetime
import re


# Chrome Options

In [5]:
options = ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-infobars')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
options.add_argument("--accept-lang=en-US,en;q=0.5")
options.add_argument("--dom-automation=disabled")
# options.binary_location = ("/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome")
if platform == 'linux' or platform == 'linux2':
    options.add_argument('--disable-dev-shm-usage')
    

In [6]:
driver = webdriver.Chrome(options=options)
 
# PAGE PARAMETERS

# keyword = "warung+kelontong"
# lat = "-6.323116"
# long = "106.870940"

keyword = "toko+pakaian"
lat = "-6.204"
long = "106.8051"

zoom = 16



# OPEN PAGE
url = f"https://www.google.com/maps/search/{keyword}/@{lat},{long},{zoom}z/data=!3m1!4b1?entry=ttu"
driver.get(url)
time.sleep(2)

# # SCROLL DOWN REPETITION : no auto stop when data end
# scrollable_list = driver.find_element(By.XPATH, "//div[contains(@role,'feed')]")
# for index, value in enumerate(range(100)): # Adjust params for custom repetition
#     print(index)
#     scrollable_list.send_keys(Keys.PAGE_DOWN)
#     time.sleep(0.5)  

# SCROLL DOWN REPETITION : auto stop when data end
scrollable_list = driver.find_element(By.XPATH, "//div[contains(@role,'feed')]")
# max_scrolls = 100  # Maximum number of scrolls to perform
max_scrolls = 5  # Maximum number of scrolls to perform

scroll_count = 0
end_condition_met = False
while not end_condition_met and scroll_count < max_scrolls:
    try:
        driver.find_element(By.CSS_SELECTOR, '.HlvSq').text
        end_condition_met = True
        print("End of Page")
    except NoSuchElementException:
        scrollable_list.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)  # Adjust this time as needed
        scroll_count += 1
        print(f"scroll_count: {scroll_count}")
print("-" * 50)

# location_list = driver.find_elements(By.CSS_SELECTOR, '.hfpxzc') # list of location  
location_list = driver.find_elements(By.XPATH, "//a[contains(@class,'hfpxzc')]")

# RECORD LIST OF LOCATION
results = []

for index, result in enumerate(location_list):
    url = result.get_attribute('href')
    
    match = re.search(r'!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)', url)
    latitude = ""
    longitude = ""
    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
    
    try:
        result.click()
    except ElementClickInterceptedException: # can't click handling
        print(result)
        print("-" * 50)
        continue
    
    time.sleep(1)
    scrollable_list.send_keys(Keys.PAGE_DOWN)
    
    try:
        phone = driver.find_element(By.XPATH, "//button[contains(@data-item-id,'phone')]").text 
    # except NoSuchElementException:
    #     phone = "" 
    except Exception as error:
        phone = "" 
        print(f"An exception was thrown!:,{error}")
    
    try:
        title = driver.find_element(By.CSS_SELECTOR, '.DUwDvf.lfPIob').text
    # except NoSuchElementException:
    #      title = ""
    except Exception as error:
        title = "" 
        print(f"An exception was thrown!:,{error}")
       
    try:
        address = driver.find_element(By.CSS_SELECTOR, '.Io6YTe.fontBodyMedium.kR99db').text
    # except NoSuchElementException:
    #      address = ""
    except Exception as error:
        address = "" 
        print(f"An exception was thrown!:,{error}")
            
    try:
        rating = driver.find_element(By.CSS_SELECTOR, '.fontDisplayLarge').text
    # except NoSuchElementException:
    #      rating = ""
    except Exception as error:
        rating = "" 
        print(f"An exception was thrown!:,{error}")
            
    try:
        reviewer = driver.find_element(By.CSS_SELECTOR, '.HHrUdb.fontTitleSmall.rqjGif').text.split()[0]
    # except NoSuchElementException:
    #      reviewer = ""
    except Exception as error:
        reviewer = ""
        print(f"An exception was thrown!:,{error}")
            
    try:
        detail = driver.find_element(By.CSS_SELECTOR, '.fontDisplayLarge').text
    # except NoSuchElementException:
    #      detail = ""
    except Exception as error:
        detail = ""
        print(f"An exception was thrown!:,{error}")
            
    print("Store Name:", title)
    print("Address:", address)
    print("Latitude:", latitude)
    print("Longitude:", longitude)
    print("Rating:", rating)
    print("Reviewer:", reviewer)
    print("Phone:", phone)
    print("URL:", url)
    print("-" * 50)
    
    results.append({
            "Store Name:": title,
            "Address": address,
            "Latitude":latitude,
            "Longitude": longitude,
            "Rating": rating,
            "Reviewer": reviewer,
            "Phone": phone,
            "URL": url
    })
    
# driver.quit()
dt_now = datetime.now().strftime('%Y%m%d_%H_%M_%S')
df_data = pd.DataFrame(results)
df_data.to_csv(f'list_toko_pakaian_tanah_abang{dt_now}.csv',index=False)


print("---DONE--")

scroll_count: 1
scroll_count: 2
scroll_count: 3
scroll_count: 4
scroll_count: 5
--------------------------------------------------
An exception was thrown!:,Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(@data-item-id,'phone')]"}
  (Session info: chrome=119.0.6045.105); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6E8E482B2+55298]
	(No symbol) [0x00007FF6E8DB5E02]
	(No symbol) [0x00007FF6E8C705AB]
	(No symbol) [0x00007FF6E8CB175C]
	(No symbol) [0x00007FF6E8CB18DC]
	(No symbol) [0x00007FF6E8CECBC7]
	(No symbol) [0x00007FF6E8CD20EF]
	(No symbol) [0x00007FF6E8CEAAA4]
	(No symbol) [0x00007FF6E8CD1E83]
	(No symbol) [0x00007FF6E8CA670A]
	(No symbol) [0x00007FF6E8CA7964]
	GetHandleVerifier [0x00007FF6E91C0AAB+3694587]
	GetHandleVerifier [0x00007FF6E921728E+4048862]
	GetHandleVerifier [0x00007FF6E920F17